In [1]:
import dlib
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np

In [2]:
detector=dlib.get_frontal_face_detector()
predictor=dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

classifier =load_model('eyes.h5')

In [3]:
class_labels = ['Closed_Eyes','Open_Eyes']

In [4]:
from pyfirmata import Arduino

cap=cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)
if __name__ == "__main__" :
    board =Arduino('COM3')
    while True:
        ret,frame=cap.read()
        if ret==False :
            break
        gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

        faces=detector(gray)

        for face in faces :
            x1,x2,y1,y2=face.left(),face.right(),face.top(),face.bottom()
            preds = 'Hossam'
            label_position = (200,200)
            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,255),3)
            cv2.putText(frame,preds,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(125,125,125),3)
            
            landmarks=predictor(gray,face)
            x11=landmarks.part(22).x
            y11=landmarks.part(22).y
            x22=landmarks.part(15).x
            y22=landmarks.part(15).y

            cv2.rectangle(frame,(x11,y11),(x22,y22),(0,0,255),3)
            roi_gray = frame[y11:y22,x11:x22]
            roi_gray = cv2.resize(roi_gray,(86,86),interpolation=cv2.INTER_AREA)


            if np.sum([roi_gray])!=0:
                roi = roi_gray.astype('float')/255.0
                roi = img_to_array(roi)
                roi = np.expand_dims(roi,axis=0)

            # make a prediction on the ROI, then lookup the class

                preds = classifier.predict(roi)[0]
                label=class_labels[preds.argmax()]

                if label=='Closed_Eyes':
                    cv2.rectangle(frame,(x11,y11),(x22,y22),(0,0,255),3)
                    cv2.putText(frame,label,(100,100),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),3)
                    board.digital[6].write(1)
                    board.digital[7].write(0)
                else :
                    cv2.rectangle(frame,(x11,y11),(x22,y22),(0,255,0),3)
                    cv2.putText(frame,label,(100,100),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
                    board.digital[7].write(1)
                    board.digital[6].write(0)
        cv2.imshow('img',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()